# IVR7 Advanced Example
This example aims to highlight the follow:
* illustrate the usage of the data-upload as well as model versioning.
* Builds a simple pickup/dropoff (similar to the basic model).
* Illustrates how to use the parent solve-request container for evaluate requests against a versioned model
  * This is a quicker manner in which to run evaluate requests against the api where a large chunk of model content is required to define the model. 
  * Send it once, then just run requests against that model and we'll handle moving the data around on the api-end.

## Requirements
This example assumes you've configured an api-key with the required services enabled (see the [portal](portal.icepack.ai) for configuration details) and that you're familiar with loading and working with protobuf models.

## Data
We're going to load sample data which has order sizes and durations in a tabular format.


In [ ]:
import pandas
df = pandas.read_csv('../../sample_data/publist_orders.csv')
print(df.head())

## Model Configuration
This model builds the ivr7-basic example.
The same dimensions (distance,time and capacity) are used as well as a handful of jobs, one vehicle cost class, one vehicle class and five vehicles.

In [ ]:
exec(open('apiHelper.py').read()) # import some api-helper classes which we've written for you.
exec(open('ivr7-model-helper.py').read()) # import some modelling helpers

api = apiHelper(modelType="ivr7-kt461v8eoaif") # set the model type to the ivr7 model

sr = ivr7_kt461v8eoaif_pb2.SolveRequest()
sr.solveType = 0 # optimise solve request

m = sr.model
m.dimensions.CopyFrom(make_distance_time_cap_dims())
m.locations.extend(make_locations(df)) #using the wrapper function
m.jobs.extend(make_job_time_cap(df, [0] * (df.shape[0]-1), range(1, df.shape[0])))
m.vehicleCostClasses.append(make_vcc_simple('vcc1', 1000, 1.001e-2, 1.001e-2, 1.001e-2, 1, 3))
m.vehicleClasses.append(make_vc_simple('vc1', 1, 1, 1, 1))
for i in range(1,5):
    m.vehicles.append( make_vehicle_cap('vehicle_' + str(i), 'vc1', 'vcc1',
                                        2000, # the vehicle capacity
                                        df['id'][0], # the start location
                                        df['id'][0], # the end location
                                        7*60,        # the start time (7AM)
                                        18*60))       # the end time  (6PM)
# at this point we have a complete model which we can run. So we can go ahead and do that!

reqId = api.Post(sr)
sol = api.Get(reqId)
t = tabulate(sr, sol)

### Configuration a cached model
At this point we've built and run a basic model. Lets now use the objects, but submit them to the api through a different mechanism.

In [ ]:
data_api = apiHelper(modelType="ivrdata-o43e0dvs78zq") # this instantiates a second model helper, 
                                            # one which works with data payloads

data_model = ivrdata_o43e0dvs78zq_pb2.CachedModel()
    
data_model.model = m.SerializeToString() # so we save the model in serialised form here into the payload


### Send the model to the api
Next we can send the data to the api. This won't solve the model, it will only stage the model on the api-end so that you can reference it in subsequent requests. We'll save the request id which then refers to the data which has been versioned.

In [ ]:
modelID = data_api.Post(data_model)
print("model id: " + modelID)

### Link the previous solve request to the versioned model on the api.
We can now delete the previous model data and create a solve request which uses the model id we've just received from the api for the model which was uploaded.

In [ ]:
sr = ivr7_kt461v8eoaif_pb2.SolveRequest()
sr.solveType = 0 # optimise solve request
sr.modelID = modelID
print(sr)
# so an empty model, but a solve-request to optimise a data-model which has been loaded

### Submit the solve request
We can now send the solve-request to the api which will solve the model which has previously been uploaded.

In [ ]:
reqId = api.Post(sr)
sol = api.Get(reqId)
t = tabulate(sr, sol)

This means that because we have a model which is versioned separately from the  solve request, we can use the solve request with the task-sequence and have that apply to a model. So lets extract the task sequence from the solved model.

### Extracting the task-sequence
The aim now is to add a task-sequence to the model. In order to do that, we need to organise the data according to vehicle (i.e. provide a task-sequence per vehicle). We can query the tabulated data for the sequence filtering out the vehicle-start and vehicle-end nodes (these are implicitly created in the model).

In addition, we don't really have a "model" because the master-data for the model is already on the api. So in this example, we can use the field on the solve-request to provide a task-sequence separately from the model.

In [ ]:
nodes = t['nodes']
del sr.routes[:]
for vid in set(nodes.vehicleId):
    tseq = ivr7_kt461v8eoaif_pb2.TaskSequence()
    tseq.vehicleId = vid
    tseq.taskId.extend(list(nodes[~nodes.taskId.str.contains('Shift') & (nodes['vehicleId']==vid)].taskId))
    sr.routes.append(tseq)

sr.solveType = 1 # lets change this to an evaluate request now.
print(sr) # so we only have a data-model reference, a task sequence, and an evaluate request

### Re-run the model
Now we can submit the evaluate request against the versioned model payload.

In [ ]:
reqId = api.Post(sr)
sol_eval = api.Get(reqId)
tab_eval = tabulate(sr, sol_eval)

In [ ]:
tab_eval['edges']

In [ ]:
tab_eval['nodes'].head()

So a note on the tabular outputs here: The python class uses the solve request to pull out the master data that was used for a particular model (such as the location x-y geocode). In this case, because there is no model in the solve request it won't be able to tack on sensible values for the x,y (nodes) and fx,fy,tx,ty (edges). This is not really a problem because we have the master data on our end so we could update these columnns if we wanted to use them for something meaningful.

### What next?
The next advanced example or course!